In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving archive (13).zip to archive (13).zip


In [ ]:
import os
os.listdir()

['.config', 'drive', 'archive (13).zip', 'sample_data']

In [ ]:
import zipfile

with zipfile.ZipFile('archive (13).zip', 'r') as zip_ref:
    zip_ref.extractall('extracted_data/')


In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers,models
from tensorflow.keras.utils import to_categorical

In [ ]:
df = 'extracted_data/'

In [ ]:
df

'extracted_data/'

In [117]:
label_mapping = {
    'Fist': 0,
    'OpenHand': 1,
    'TwoFinger': 2,
}

In [118]:
images = []
labels = []

In [119]:
import cv2

for i in os.listdir('extracted_data/'):
    dir = os.path.join('extracted_data/', i)
    if i in label_mapping:
        label = label_mapping[i]
        for image_name in os.listdir(dir):
            image_path = os.path.join(dir, image_name)
            img = cv2.imread(image_path)
            img = cv2.resize(img, (64, 64))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            images.append(img)
            labels.append(label)

images = np.array(images)
labels = np.array(labels)

if len(images) != len(labels):
      print("Error: Number of images and labels do not match.")

images = images.reshape(-1,64, 64, 1)

In [120]:
images.shape

(6018, 64, 64, 1)

In [121]:
labels.shape

(6018,)

In [122]:
num_classes = len(label_mapping)

In [123]:
import tensorflow as tf
labels = tf.keras.utils.to_categorical(labels, num_classes=num_classes)

In [124]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [125]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [126]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

In [127]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [128]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10
151/151 [==============================] - 26s 167ms/step - loss: 0.2883 - accuracy: 0.8793 - val_loss: 0.1061 - val_accuracy: 0.9585
Epoch 2/10
151/151 [==============================] - 27s 178ms/step - loss: 0.0227 - accuracy: 0.9944 - val_loss: 0.0272 - val_accuracy: 0.9925
Epoch 3/10
151/151 [==============================] - 27s 176ms/step - loss: 0.0197 - accuracy: 0.9938 - val_loss: 0.0041 - val_accuracy: 0.9992
Epoch 4/10
151/151 [==============================] - 25s 164ms/step - loss: 9.2340e-04 - accuracy: 1.0000 - val_loss: 0.0025 - val_accuracy: 0.9992
Epoch 5/10
151/151 [==============================] - 24s 158ms/step - loss: 3.2826e-04 - accuracy: 1.0000 - val_loss: 0.0024 - val_accuracy: 0.9992
Epoch 6/10
151/151 [==============================] - 28s 187ms/step - loss: 2.2086e-04 - accuracy: 1.0000 - val_loss: 0.0022 - val_accuracy: 0.9992
Epoch 7/10
151/151 [==============================] - 24s 157ms/step - loss: 1.5821e-04 - accuracy: 1.0000 - val_loss:

In [129]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

38/38 [==============================] - 1s 37ms/step - loss: 0.0019 - accuracy: 0.9992
Test accuracy: 0.9991694092750549
